<a href="https://colab.research.google.com/github/sarahpursley/dsci551-assignment2/blob/main/GPT_2_Text_Generating_Model_w_GPU_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

**GPU:** Use either a Nvidia T4 GPU or an Nvidia K80 GPU and verify which GPU.


**SE Types:** Reconnaissance, Social Engineering, Malware, Phishing


In [1]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

!nvidia-smi

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Thu Apr  8 02:12:07 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4    

**download_gpt2: only run ONCE to download**

**file upload from local machine: only upload once into files**

In [6]:
import os
from google.colab import files

os.chdir('/content')
uploaded = files.upload()

Saving fraudulent_emails_v2.json to fraudulent_emails_v2.json


In [37]:
import json
import re
messages = json.load(open('fraudulent_emails_v2.json',))

recon_train = []
se_train = []
phish_train = []
mal_train = []

whitespace_ptrn = re.compile(r'\s')

for message in messages:
    if 'se_tag' in message:
        ttr_dict = message['TTR_text']
        t = ttr_dict['ttr_text']
        norm_content = whitespace_ptrn.sub(' ', t).encode('utf-8')

        if t != '':
            if message['se_tag'] == 'recon':
                t = t.lstrip().strip()
                recon_train.append(norm_content.decode("utf-8"))

            if message['se_tag'] == 'social_engineering':
                se_train.append(norm_content.decode("utf-8"))

            if message['se_tag'] == 'phishing':
                phish_train.append(norm_content.decode("utf-8"))

            elif message['se_tag'] == 'malware':
                mal_train.append(norm_content.decode("utf-8"))

            else:
                continue
    else:
        print('hi')
    

print("Recon train:", len(recon_train))
print("SE messages:", len(se_train))
print("Phishing messages: ", len(phish_train))
print("Malware messages: ", len(mal_train))

Recon train: 446
SE messages: 1706
Phishing messages:  1447
Malware messages:  79


['URGENT ASSISTANCE /RELATIONSHIP (P) Dear Friend, I am Mr. Ben Suleman a custom officer and work as Assistant controller of the Customs and Excise department Of the Federal Ministry of Internal Affairs stationed at the Murtala Mohammed International Airport, Ikeja, Lagos-Nigeria. After the sudden death of the former Head of state of Nigeria General Sanni Abacha on June 8th 1998 his aides and immediate members of his family were arrested while trying to escape from Nigeria in a Chartered jet to Saudi Arabia with 6 trunk boxes Marked "Diplomatic Baggage". Acting on a tip-off as they attempted to board the Air Craft,my officials carried out a thorough search on the air craft and discovered that the 6 trunk boxes contained foreign currencies amounting to US$197,570,000.00(One Hundred and  Ninety-Seven Million Five Hundred Seventy Thousand United States Dollars). I declared only (5) five boxes to the government and withheld one (1) in my custody containing the sum of (US$30,000,000.00) Thi

In [38]:
# write out text files with messages by attacker type
# use to generate later
with open("recon_train.txt", "w") as output:
    output.write(str(recon_train))

with open("se_train.txt", "w") as output:
    output.write(str(se_train))

with open("phish_train.txt", "w") as output:
    output.write(str(phish_train))

with open("mal_train.txt", "w") as output:
    output.write(str(mal_train)) 

# Finetune GPT2 Model

## Recon Model


In [5]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

!nvidia-smi

gpt2.download_gpt2(model_name="124M")

ERROR: Operation cancelled by user
Thu Apr  8 02:55:15 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    34W /  70W |   8484MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                     

Fetching checkpoint: 1.05Mit [00:00, 375Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 4.41Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 295Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001:   0%|                 | 2.10M/498M [00:00<01:07, 7.37Mit/s]


KeyboardInterrupt: ignored

In [ ]:
file_name = "recon_train.txt"
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=1000,
              restore_from='fresh',
              run_name='recon',
              print_every=10,
              sample_every=100,
              save_every=200
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:01<00:00,  1.56s/it]


dataset has 284886 tokens
Training...
[10 | 28.46] loss=3.40 avg=3.40
[20 | 50.25] loss=2.66 avg=3.03
[30 | 72.70] loss=2.81 avg=2.96
[40 | 96.07] loss=2.70 avg=2.89
[50 | 120.56] loss=2.87 avg=2.89
[60 | 144.53] loss=2.50 avg=2.82
[70 | 167.98] loss=2.44 avg=2.76
[80 | 191.71] loss=1.73 avg=2.63
[90 | 215.60] loss=1.96 avg=2.55
[100 | 239.31] loss=1.77 avg=2.47
======== SAMPLE 1 ========
 assist to furnish the needed particulars, and the Attorney-at-law will present the relevant documents and affidavits of the said parties to back up his proposition.All parties will be provided with the following documents/firma- notes as proof of the said amount: (1) In-kind gratuities of US$100.00 (2) In-kind gratuities of US$5,000,000 (3) In-kind gratuities of US$500,000.00 (4) In-kind gratuities of US$50,000.00 (5) In-kind gratuities of US$250000.00, and all the required documentation will be used in the judgement of the said Attorney.    I will be contacting you to assist in this business, in cas

In [ ]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

!nvidia-smi

gpt2.download_gpt2(model_name="124M")

In [2]:
sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, run_name='recon')
gpt2.generate(sess, run_name='recon',prefix = 'FROM')

Loading checkpoint checkpoint/recon/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/recon/model-1000
FROM OUR SON, AS I AM TO MAKE THE TRANSFER TO YOUR COUNTRY BEFORE THE PROPOSAL IS OVER.  YOUR TRANSFER AND REPLY WILL BE GIVEN TO YOU PER INDICATED MONTHLY.   I AM CONTACTING YOU BECAUSE OF THE NEED TO INVOLVE A FOREIGNER WITH A FOREIGN ACCOUNT THAT WILL STAND-INAS THE NEXT OF KIN TO MR.SESAY BAKER. WITH THE NEED TO INVOLVE A FOREIGNER WITH A FOREIGN ACCOUNT THAT WILL STAND-INAS THE NEXT OF KIN TO MR.SESAY  SIR,   SO THAT I CAN INTRODUCE YOU TO MY FAMILY AS OUR FOREIGNERS,   TO INVOLVE A FOREIGNER AS OUR FOREIGN BONAFIDE OF MOVING THE TRUSTED OPPORTUNITY  WILL BE THE SHARE OF 75% OF THE TOTAL SUM AS GRATIFICATION,  BUT WILL BE REQUIRED FOR TRANSFER INTO YOUR ACCOUNT AS WEIGHT INCORPORATED WITH YOUR  ABIDU CUSTOMER.   I AM CONTACTING YOU BECAUSE OF THE NEED TO INVOLVE A FOREIGNER WITH A FOREIGN ACCOUNT THAT WILL STAND-INAS THE NEXT OF KIN TO MR.SESAY BAKER.   IS THIS NEED

In [5]:
recon_gen_file = 'gpt2_gentext_recon.txt'

gpt2.generate_to_file(sess, destination_path = recon_gen_file, model_name='124M',                      
                      length = 500,
                      temperature = 0.5, nsamples = 200, batch_size = 20)

In [6]:
files.download(recon_gen_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Social Engineering Model


In [ ]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

!nvidia-smi

gpt2.download_gpt2(model_name="124M")

In [4]:
# se model finetune
file_name = "se_train.txt"
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=1000,
              restore_from='fresh',
              run_name='se',
              print_every=10,
              sample_every=100,
              save_every=250
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:05<00:00,  5.09s/it]


dataset has 1062473 tokens
Training...
[10 | 26.46] loss=3.32 avg=3.32
[20 | 48.09] loss=3.25 avg=3.29
[30 | 70.05] loss=2.58 avg=3.05
[40 | 92.21] loss=2.56 avg=2.92
[50 | 114.75] loss=2.97 avg=2.93
[60 | 137.29] loss=3.20 avg=2.98
[70 | 159.90] loss=2.64 avg=2.93
[80 | 182.73] loss=2.59 avg=2.88
[90 | 205.73] loss=2.72 avg=2.87
[100 | 228.91] loss=2.15 avg=2.79
======== SAMPLE 1 ========
 in U.N.R.R.E is based on this money. I am the only son and all this money is from my late father because he is from Nigeria and I had been looking forward to having a child from his country due to his close relationship with my father. My father had a few very prosperous properties in South Asia, Europe and Asia-Pacific (Sierra Leone) as well as a large fortune in Europe and Asia-Pacific. This is why I want you to help us to collect this money from your country. I have told my mother that I will give her an application for transfer to the West African countries, because I want to go back to my fathe

In [1]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

!nvidia-smi

#gpt2.download_gpt2(model_name="124M")

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Thu Apr  8 04:15:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4    

In [2]:
sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, run_name='se')
gpt2.generate(sess, run_name='se',prefix = 'FROM')

Loading checkpoint checkpoint/se/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/se/model-1000
FROM us.  You will be paid 30% of the total sum for your efforts/assistance,while 10% has been set aside for any expenses that might be incurred or any other expenses that you may incure in the process of this transaction.  I look forward to your fullest co-operation.  Yours Faithfully,  Mr.Kofi Nzanga. ', "Very urgent Dear,  Good day to you, I pray that this contact should come to you as a  positive  and save you from a lifetime of sorrow and confusion.I am from the republic of Sierria-Leone but now seeking  political asylum in Senegal- Africa. I got your contact from the website under  "contact me" section of internet friday night before i lost my husband and my 3 daughters in the plane crash in dakar senegal.  I got your contact through network  as i am still in the asylum camp and i have decided to contact you after  much personal searching and hard work.  I am Barrister J

In [3]:
se_gen_file = 'gpt2_gentext_se.txt'

gpt2.generate_to_file(sess, destination_path = se_gen_file, model_name='124M',                      
                      length = 500,
                      temperature = 0.5, nsamples = 200, batch_size = 20)

In [4]:
files.download(se_gen_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Phishing Model


In [1]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

!nvidia-smi

gpt2.download_gpt2(model_name="124M")

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Thu Apr  8 03:11:37 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4    

Fetching checkpoint: 1.05Mit [00:00, 427Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 5.53Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 533Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:51, 9.64Mit/s]
Fetching model.ckpt.index: 1.05Mit [00:00, 773Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 8.76Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 6.74Mit/s]


In [2]:
file_name = "phish_train.txt"
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=1000,
              restore_from='fresh',
              run_name='phish',
              print_every=10,
              sample_every=100,
              save_every=250
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:04<00:00,  4.85s/it]


dataset has 1026585 tokens
Training...
[10 | 27.13] loss=3.31 avg=3.31
[20 | 49.64] loss=2.85 avg=3.08
[30 | 72.21] loss=2.98 avg=3.05
[40 | 94.97] loss=2.71 avg=2.96
[50 | 117.87] loss=2.58 avg=2.88
[60 | 140.97] loss=2.62 avg=2.84
[70 | 164.17] loss=2.80 avg=2.83
[80 | 187.49] loss=2.10 avg=2.74
[90 | 210.89] loss=2.46 avg=2.70
[100 | 234.42] loss=2.22 avg=2.65
======== SAMPLE 1 ========
 leaders to assist the victims,  and send it to your  country where the  fund-sharing will be finalized in your favour.  If the money has not been paid properly, the money will  be forfeited or forfeited if the  fund shall not be paid up  within a certain amount of time.  This  transaction would be most profitable for both of us because  I  will be working as an  auditor and the money is secured  by an  award-sharing agreement.  I  await your full  response .                     _________________________________________________________________________ DETAILS  ---------------------                   

In [1]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

!nvidia-smi

#gpt2.download_gpt2(model_name="124M")

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Thu Apr  8 05:04:57 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4    

In [3]:
sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, run_name='phish')
gpt2.generate(sess, run_name='phish',prefix = 'FROM')

Loading checkpoint checkpoint/phish/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/phish/model-1000
FROM THE GOVERNMENT DEVELOPMENT BANK OF NIGERIA (NCBN)  I WISH TO  COMMENCE THIS TRANSACTION WITH YOU IN WHICH I AM AND DO NOT  HAVE A NEXT OF KIN OR KIND BREED. THIS IS TO PRESENT ME AS A FOREIGNER WHO  DIED ON MY LAST MISSION INTO AFRICA IN THE YEAR 1998 AND I HAVE BEEN  INFORMED BY MY COMPUTING AND REQUIRING OFFICERS IN YOUR  COMPANY THAT I WILL BE EXPECTED TO PROVIDE FOR ME AFTER MY DEPOSIT WITH  YOUR COMPANY, AFTER WHICH I WILL VISIT YOU AND REMIT THE  MONEY TO YOUR COMPANY FOR SHARING, SUBSEQUENT  ACCOUNT SERVICE OR OTHER MODALITIES.  MY DEAR FRIEND, I WILL NEED YOUR ASSISTANCE TO  INFORM YOU THAT THIS TRANSACTION IS 100% RISK FREE. YOU SHOULD  ALSO RESEND ME YOUR CONSENT IMMEDIATELY TO ALLOW ME THE  NECESSARY 10% OF THE TOTAL SUM TO BE USED FOR THE  DISCRETION AND EXPENSES TAIRING.  I AWAIT YOUR URGENT REPLY.  PLEASE REPLY ME AS SOON AS POSSIBLE.  PLEASE INDICATE 

In [4]:
phish_gen_file = 'gpt2_gentext_phish.txt'

gpt2.generate_to_file(sess, destination_path = phish_gen_file, model_name='124M',                      
                      length = 500,
                      temperature = 0.5, nsamples = 200, batch_size = 20)

In [5]:
files.download(phish_gen_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Malware Model


In [ ]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

!nvidia-smi

gpt2.download_gpt2(model_name="124M")

In [2]:
# malware model finetune
file_name = "mal_train.txt"
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=1000,
              restore_from='fresh',
              run_name='mal',
              print_every=10,
              sample_every=100,
              save_every=250
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:00<00:00,  2.51it/s]


dataset has 56625 tokens
Training...
[10 | 27.48] loss=2.69 avg=2.69
[20 | 49.14] loss=2.78 avg=2.74
[30 | 71.04] loss=1.79 avg=2.42
[40 | 93.26] loss=1.94 avg=2.30
[50 | 115.80] loss=1.08 avg=2.05
[60 | 138.46] loss=1.58 avg=1.97
[70 | 161.19] loss=1.27 avg=1.87
[80 | 184.12] loss=1.36 avg=1.80
[90 | 207.21] loss=0.72 avg=1.68
[100 | 230.43] loss=0.42 avg=1.55
======== SAMPLE 1 ========
 hundred percent of both the total sum and any interest which might be accrued on it at maturity before the maturity date. This deal is sometimes termed the Chase  Direct offer.  The money is already on its way to you in a checabdn form, from which it is waiting  for our Diplomatic Security and Finance Representative to reach here in Dubai.  In case you are not familiar with him or his  contact email, I recommend watching this video:  http://www.cnn.com/2010/WORLD/02/02/uk/politics/the-second-in-second-in-second-episode-1.  http://www.cnn.com/2002/WORLD/2002/WORLD/europe/  So get down to business now, 

In [1]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

!nvidia-smi

#gpt2.download_gpt2(model_name="124M")

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Thu Apr  8 05:47:07 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4    

In [2]:
sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, run_name='mal')
gpt2.generate(sess, run_name='mal',prefix = 'FROM')

Loading checkpoint checkpoint/mal/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/mal/model-1000
FROM THE RELEASE:From the Financial Institution:  My name is Mrs.Monica Martins, the wife of late Mr.Zulu Martins, of Liberia.We are in our possession vital approved papers to invoice Mr.Zulu Martins as the next of kin to Lassalle Dembo, an Iraqi refugee ,after which time the couple will not be related. After this Dembo died, and new laws which limited the number of refugees we could accept was introduced, lassalle decided to ban all biological materials and financial records of all Liberians within 14 working days beginning with his last known addresses. This has made my husband the next of kin in lawless Africa, and he has been since de facto the head of state for the past four decades now.  I have in mind that we keep this money for as personal protection since I cannot lodge the required documents with the relevant immigration department.  However, upon due consultation 

In [3]:
mal_gen_file = 'gpt2_gentext_mal.txt'

gpt2.generate_to_file(sess, destination_path = mal_gen_file, model_name='124M',                      
                      length = 500,
                      temperature = 0.5, nsamples = 200, batch_size = 20)

In [4]:
files.download(mal_gen_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Force-kill VM

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [ ]:
!kill -9 -1

# LICENSE

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

##  Train a GPT-2 Text-Generating Model w/ GPU For Free 

by [Max Woolf](http://minimaxir.com)

*Last updated: February 14th, 2021*

Retrain an advanced text generating neural network on any text dataset **for free on a GPU using Collaboratory** using `gpt-2-simple`!

For more about `gpt-2-simple`, you can visit [this GitHub repository](https://github.com/minimaxir/gpt-2-simple). You can also read my [blog post](https://minimaxir.com/2019/09/howto-gpt2/) for more information how to use this notebook!


# Unused Code

In [ ]:
gpt2.generate(sess, run_name='run1')

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.